In [ ]:
#!python -m spacy download en_core_web_lg

In [ ]:
#!python -m spacy download en_core_web_trf

In [ ]:
#!python -m coreferee install en

In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
torch.cuda.device_count()

1

In [4]:
print(torch.version.cuda)

11.6


In [5]:
import coreferee, spacy

nlp = spacy.load('en_core_web_trf')
nlp.add_pipe('coreferee')

0: Tom(0), his(5), he(8), he(11)
1: Texas(3), it(13)


In [27]:
text = "Tom was in Texas with his wife and he said that he liked it"
#text = "Although he was very busy with his work, Peter had had enough of it. He and his wife decided they needed a holiday. They travelled to Spain because they loved the country very much."


In [28]:
#Basic usage of coreferee
doc = nlp(text)
doc._.coref_chains.print()

0: Tom(0), his(5), he(8), he(11)
1: Texas(3), it(13)


In [31]:
print(doc._.coref_chains.resolve(doc[13]))

[Texas]


In [32]:
print(doc._.coref_chains.resolve(doc[3]))

None


In [ ]:
# Find parent/original entity and resolve other references to it.
# In above example 'it' resolved to 'Texas' but 'Texas' resolved to None because Texas is original entity.
# Basic logic to get resolved text:
#1. Get all token words
#2. Check resolution of each token 
#3. If it is None,it means it's original entity or there is nothing to resolve, so ignore None
#4. If it is not None,then replcae token with resolved/original entity
#5. Combine all tokens back to get resolved text

In [24]:
doc = nlp(text)
# Get token list
tok_list = list(token.text_with_ws for token in doc)
print(tok_list)
for index, _ in enumerate(tok_list):
    #Check resolution of each token 
    if None != doc._.coref_chains.resolve(doc[index]):
        new_token = ""
        #If it is not None,then replcae token with resolved/original entity
        for resolved_token in doc._.coref_chains.resolve(doc[index]):
            new_token = new_token + resolved_token.text + " "
        tok_list[index] = new_token
resolved_text = "".join(tok_list)

['Tom ', 'was ', 'in ', 'Texas ', 'with ', 'his ', 'wife ', 'and ', 'he ', 'said ', 'that ', 'he ', 'liked ', 'it']


In [22]:
resolved_text

'Tom was in Texas with Tom wife and Tom said that wife liked Texas '